<a href="https://colab.research.google.com/github/savinkova-mariia/NLP/blob/main/NLP_lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from os import getcwd
import numpy as np

filePath = f"{getcwd()}/content/utils.py"
nltk.data.path.append(filePath)
from utils import build_freqs

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
df = pd.read_table('/content/reviews.tsv')

df_new = df.drop(df[(df["rating"] == 0.0)].index)

In [70]:
pos_train, pos_test, neg_train, neg_test = train_test_split(df_new.review_text, df_new.rating, test_size=0.2)

In [74]:
train_y = np.append(np.ones((len(pos_train), 1)), np.zeros((len(neg_train), 1)), axis=0)
test_y = np.append(np.ones((len(pos_test), 1)), np.zeros((len(neg_test), 1)), axis=0)

#freqs = build_freqs(pos_train, train_y)

TypeError: ignored